
# Formulário de Avaliação Inicial do Corredor

Este notebook funciona como um **formulário interativo** para coleta de dados de um corredor.  
Ao preencher os campos e clicar em **Salvar dados e classificar**, o notebook irá:

1. Construir um objeto Python seguindo a **estrutura do JSON Schema** definido previamente.
2. Salvar os dados em um arquivo `corredor_dados.json` no diretório atual.
3. Gerar um **rótulo automático** de classificação do corredor:  
   - `iniciante`  
   - `intermediario`  
   - `avancado`  

A classificação é baseada principalmente em:

- `volume_semanal_km`
- `melhor_5k_segundos`

> **Obs.:** Campos não preenchidos em tempos de prova serão salvos como `null` no JSON.


In [ ]:

import json
from ipywidgets import (
    VBox, HBox, Label, BoundedIntText, BoundedFloatText,
    Dropdown, Checkbox, Button, Output
)
from IPython.display import display, clear_output

STYLE_FULL = {'description_width': 'initial'}

LAYOUT_NUMERIC = {'min_width': '260px'}
LAYOUT_TIME = {'min_width': '220px'}


def mm_ss_to_seconds(min_widget, sec_widget):
    m = min_widget.value
    s = sec_widget.value
    if m is None and s is None:
        return None
    if m is None:
        m = 0
    if s is None:
        s = 0
    return int(m) * 60 + int(s)

def hh_mm_ss_to_seconds(h_widget, m_widget, s_widget):
    h = h_widget.value
    m = m_widget.value
    s = s_widget.value
    if h is None and m is None and s is None:
        return None
    h = 0 if h is None else h
    m = 0 if m is None else m
    s = 0 if s is None else s
    return int(h) * 3600 + int(m) * 60 + int(s)

def classificar_corredor(volume_semanal_km, melhor_5k_segundos):
    nivel_por_volume = None
    if volume_semanal_km is None:
        nivel_por_volume = None
    elif volume_semanal_km < 20:
        nivel_por_volume = "iniciante"
    elif 20 <= volume_semanal_km <= 40:
        nivel_por_volume = "intermediario"
    else:
        nivel_por_volume = "avancado"

    nivel_por_5k = None
    if melhor_5k_segundos is not None:
        if melhor_5k_segundos > 1800:
            nivel_por_5k = "iniciante"
        elif 1200 < melhor_5k_segundos <= 1800:
            nivel_por_5k = "intermediario"
        else:
            nivel_por_5k = "avancado"

    candidatos = [nivel_por_volume, nivel_por_5k]
    candidatos = [c for c in candidatos if c is not None]
    if not candidatos:
        return "iniciante"

    if "avancado" in candidatos:
        return "avancado"
    if "intermediario" in candidatos:
        return "intermediario"
    return "iniciante"

idade_anos = BoundedIntText(description="Idade (anos)", min=10, max=90, value=30, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
sexo = Dropdown(description="Sexo", options=[("Masculino", "masculino"),
                                             ("Feminino", "feminino"),
                                             ("Outro", "outro")], value="masculino", style=STYLE_FULL)
altura_cm = BoundedFloatText(description="Altura (cm)", min=100, max=230, value=175.0, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
peso_kg = BoundedFloatText(description="Peso (kg)", min=30, max=200, value=70.0, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
profissao_atividade = Dropdown(
    description="Atividade profissional (nível de atividade)",
    options=[("Sedentário", "sedentario"),
             ("Moderadamente ativo", "moderadamente_ativo"),
             ("Altamente ativo", "altamente_ativo")],
    value="moderadamente_ativo",
    style=STYLE_FULL
)
horas_sono_noite = BoundedFloatText(description="Horas de sono por noite", min=0, max=14, value=7.0, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
estresse_diario = BoundedIntText(description="Nível de estresse diário (0-10)", min=0, max=10, value=5, style=STYLE_FULL, layout=LAYOUT_NUMERIC)

box_identificacao = VBox([
    Label("1. Identificação do atleta"),
    idade_anos, sexo, altura_cm, peso_kg,
    profissao_atividade, horas_sono_noite, estresse_diario
])

tempo_corrida_meses = BoundedIntText(description="Tempo total que corre (em meses)", min=0, max=600, value=6, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
volume_semanal_km = BoundedFloatText(description="Volume semanal atual de corrida (km)", min=0, max=300, value=20.0, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
treinos_semana = BoundedIntText(description="Número de treinos de corrida por semana", min=0, max=14, value=3, style=STYLE_FULL, layout=LAYOUT_NUMERIC)

ritmo_facil_seg_km = BoundedIntText(description="Ritmo médio de corrida fácil (segundos por km)", min=120, max=900, value=360, style=STYLE_FULL, layout=LAYOUT_TIME)
ritmo_moderado_seg_km = BoundedIntText(description="Ritmo médio de corrida moderada (segundos por km)", min=120, max=900, value=330, style=STYLE_FULL, layout=LAYOUT_TIME)

melhor_5k_min = BoundedIntText(description="Melhor tempo nos 5 km (minutos)", min=0, max=59, value=25, style=STYLE_FULL, layout=LAYOUT_TIME)
melhor_5k_sec = BoundedIntText(description="Melhor tempo nos 5 km (segundos)", min=0, max=59, value=0, style=STYLE_FULL, layout=LAYOUT_TIME)

melhor_10k_min = BoundedIntText(description="Melhor tempo nos 10 km (minutos)", min=0, max=180, value=50, style=STYLE_FULL, layout=LAYOUT_TIME)
melhor_10k_sec = BoundedIntText(description="Melhor tempo nos 10 km (segundos)", min=0, max=59, value=0, style=STYLE_FULL, layout=LAYOUT_TIME)

melhor_21k_h = BoundedIntText(description="Melhor tempo na meia maratona (horas)", min=0, max=10, value=1, style=STYLE_FULL, layout=LAYOUT_TIME)
melhor_21k_m = BoundedIntText(description="Melhor tempo na meia maratona (minutos)", min=0, max=59, value=50, style=STYLE_FULL, layout=LAYOUT_TIME)
melhor_21k_s = BoundedIntText(description="Melhor tempo na meia maratona (segundos)", min=0, max=59, value=0, style=STYLE_FULL, layout=LAYOUT_TIME)

melhor_42k_h = BoundedIntText(description="Melhor tempo na maratona (horas)", min=0, max=10, value=3, style=STYLE_FULL, layout=LAYOUT_TIME)
melhor_42k_m = BoundedIntText(description="Melhor tempo na maratona (minutos)", min=0, max=59, value=30, style=STYLE_FULL, layout=LAYOUT_TIME)
melhor_42k_s = BoundedIntText(description="Melhor tempo na maratona (segundos)", min=0, max=59, value=0, style=STYLE_FULL, layout=LAYOUT_TIME)

provas_12_meses = Dropdown(
    description="Participou de provas nos últimos 12 meses?",
    options=[("Não", False), ("Sim", True)],
    value=False,
    style=STYLE_FULL
)

box_historico_corrida = VBox([
    Label("2. Histórico de corrida"),
    tempo_corrida_meses,
    volume_semanal_km,
    treinos_semana,
    ritmo_facil_seg_km,
    ritmo_moderado_seg_km,
    HBox([melhor_5k_min, melhor_5k_sec]),
    HBox([melhor_10k_min, melhor_10k_sec]),
    HBox([melhor_21k_h, melhor_21k_m, melhor_21k_s]),
    HBox([melhor_42k_h, melhor_42k_m, melhor_42k_s]),
    provas_12_meses
])

lesao_tendinite = Checkbox(description="Histórico de tendinite", value=False, style=STYLE_FULL)
lesao_fascite = Checkbox(description="Histórico de fascite plantar", value=False, style=STYLE_FULL)
lesao_itbs = Checkbox(description="Histórico de síndrome da banda iliotibial (ITBS)", value=False, style=STYLE_FULL)
lesao_dor_lombar = Checkbox(description="Histórico de dor lombar relacionada à corrida", value=False, style=STYLE_FULL)
lesao_fratura_estresse = Checkbox(description="Histórico de fratura por estresse", value=False, style=STYLE_FULL)
lesao_outra = Checkbox(description="Outro tipo de lesão relevante", value=False, style=STYLE_FULL)

frequencia_lesoes_ano = BoundedIntText(description="Número de episódios de lesão por ano", min=0, max=20, value=0, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
condicao_medica = Dropdown(
    description="Principal condição médica relevante",
    options=[("Nenhuma", "nenhuma"),
             ("Cardíaca", "cardiaca"),
             ("Respiratória", "respiratoria"),
             ("Metabólica", "metabolica"),
             ("Outra", "outra")],
    value="nenhuma",
    style=STYLE_FULL
)
usa_medicacao = Dropdown(
    description="Uso atual de medicação contínua",
    options=[("Não", False), ("Sim", True)],
    value=False,
    style=STYLE_FULL
)

box_lesoes_saude = VBox([
    Label("3. Histórico de lesões e saúde"),
    HBox([lesao_tendinite, lesao_fascite, lesao_itbs]),
    HBox([lesao_dor_lombar, lesao_fratura_estresse, lesao_outra]),
    frequencia_lesoes_ano,
    condicao_medica,
    usa_medicacao
])

fc_reposo_bpm = BoundedIntText(description="Frequência cardíaca de repouso (bpm)", min=30, max=120, value=60, style=STYLE_FULL, layout=LAYOUT_TIME)
fc_max_bpm = BoundedIntText(description="Frequência cardíaca máxima estimada/medida (bpm)", min=120, max=230, value=190, style=STYLE_FULL, layout=LAYOUT_TIME)

teste_1k_min = BoundedIntText(description="Teste de 1 km forte (minutos)", min=0, max=59, value=4, style=STYLE_FULL, layout=LAYOUT_TIME)
teste_1k_sec = BoundedIntText(description="Teste de 1 km forte (segundos)", min=0, max=59, value=0, style=STYLE_FULL, layout=LAYOUT_TIME)

teste_3k_min = BoundedIntText(description="Teste de 3 km forte (minutos)", min=0, max=59, value=14, style=STYLE_FULL, layout=LAYOUT_TIME)
teste_3k_sec = BoundedIntText(description="Teste de 3 km forte (segundos)", min=0, max=59, value=0, style=STYLE_FULL, layout=LAYOUT_TIME)

teste_5k_min = BoundedIntText(description="Teste de 5 km forte (minutos)", min=0, max=59, value=25, style=STYLE_FULL, layout=LAYOUT_TIME)
teste_5k_sec = BoundedIntText(description="Teste de 5 km forte (segundos)", min=0, max=59, value=0, style=STYLE_FULL, layout=LAYOUT_TIME)

disp_fc = Checkbox(description="Disponibilidade de monitorização de frequência cardíaca", value=True, style=STYLE_FULL)
disp_potencia = Checkbox(description="Disponibilidade de medição de potência de corrida", value=False, style=STYLE_FULL)
disp_cadencia = Checkbox(description="Disponibilidade de medição de cadência de corrida", value=True, style=STYLE_FULL)
disp_gps = Checkbox(description="Disponibilidade de registro de treino com GPS", value=True, style=STYLE_FULL)

box_dados_fisio = VBox([
    Label("4. Dados fisiológicos e testes"),
    fc_reposo_bpm,
    fc_max_bpm,
    HBox([teste_1k_min, teste_1k_sec]),
    HBox([teste_3k_min, teste_3k_sec]),
    HBox([teste_5k_min, teste_5k_sec]),
    HBox([disp_fc, disp_potencia, disp_cadencia, disp_gps])
])

objetivo_principal = Dropdown(
    description="Objetivo principal com a corrida",
    options=[("Saúde geral", "saude"),
             ("Performance em provas", "performance"),
             ("Emagrecimento", "emagrecimento"),
             ("Prova específica", "prova_especifica")],
    value="performance",
    style=STYLE_FULL
)
meta_distancia = Dropdown(
    description="Distância-alvo principal para as provas",
    options=[("5 km", "5k"),
             ("10 km", "10k"),
             ("21 km (meia maratona)", "21k"),
             ("42 km (maratona)", "42k"),
             ("Outra distância", "outra")],
    value="10k",
    style=STYLE_FULL
)
reducao_tempo_min = BoundedFloatText(description="Redução de tempo desejada em provas (minutos)", min=0, max=120, value=0.0, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
data_proxima_prova = Dropdown(
    description="Data da próxima prova-alvo (exemplo de formato)",
    options=[("Não definida", ""), ("Exemplo: 2025-12-31", "2025-12-31")],
    value="",
    style=STYLE_FULL
)
motivacao_0_10 = BoundedIntText(description="Nível de motivação atual para treinar (0-10)", min=0, max=10, value=8, style=STYLE_FULL, layout=LAYOUT_NUMERIC)

box_objetivos = VBox([
    Label("5. Objetivos do atleta"),
    objetivo_principal,
    meta_distancia,
    reducao_tempo_min,
    data_proxima_prova,
    motivacao_0_10
])

dias_treino_semana = BoundedIntText(description="Dias disponíveis para treinar corrida por semana", min=0, max=14, value=4, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
tempo_max_sessao_min = BoundedFloatText(description="Tempo máximo disponível por sessão de treino (minutos)", min=0, max=300, value=60.0, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
acesso_academia = Checkbox(description="Possui acesso regular a academia/musculação", value=True, style=STYLE_FULL)
preferencia_treino = Dropdown(
    description="Preferência principal de tipo de treino",
    options=[("Treinos curtos", "curto"),
             ("Treinos longos", "longo"),
             ("Treinos intervalados", "intervalado"),
             ("Treinos de limiar", "limiar"),
             ("Sem preferência definida", "sem_preferencia")],
    value="sem_preferencia",
    style=STYLE_FULL
)
aderencia_historica = Dropdown(
    description="Histórico de aderência a planos de treino",
    options=[("Alta aderência", "alta"), ("Média aderência", "media"), ("Baixa aderência", "baixa")],
    value="alta",
    style=STYLE_FULL
)

box_restricoes = VBox([
    Label("6. Restrições e disponibilidade"),
    dias_treino_semana,
    tempo_max_sessao_min,
    acesso_academia,
    preferencia_treino,
    aderencia_historica
])

nivel_percebido = BoundedIntText(description="Nível percebido de condicionamento atual (0-10)", min=0, max=10, value=5, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
disponibilidade_energetica = BoundedIntText(description="Percepção de energia diária disponível para treinar (0-10)", min=0, max=10, value=7, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
tolerancia_intenso = BoundedIntText(description="Tolerância percebida a treinos intensos (0-10)", min=0, max=10, value=6, style=STYLE_FULL, layout=LAYOUT_NUMERIC)
contraindicacao_alta_intensidade = Checkbox(description="Existe contraindicação médica para treinos de alta intensidade", value=False, style=STYLE_FULL)

box_classificacao_auto = VBox([
    Label("7. Classificação automática - percepções do atleta"),
    nivel_percebido,
    disponibilidade_energetica,
    tolerancia_intenso,
    contraindicacao_alta_intensidade
])

botao_salvar = Button(description="Salvar dados e classificar corredor", button_style="success")
output = Output()

def on_salvar_clicked(b):
    with output:
        clear_output()
        melhor_5k_segundos = mm_ss_to_seconds(melhor_5k_min, melhor_5k_sec)
        melhor_10k_segundos = mm_ss_to_seconds(melhor_10k_min, melhor_10k_sec)
        melhor_21k_segundos = hh_mm_ss_to_seconds(melhor_21k_h, melhor_21k_m, melhor_21k_s)
        melhor_42k_segundos = hh_mm_ss_to_seconds(melhor_42k_h, melhor_42k_m, melhor_42k_s)

        teste_1k_segundos = mm_ss_to_seconds(teste_1k_min, teste_1k_sec)
        teste_3k_segundos = mm_ss_to_seconds(teste_3k_min, teste_3k_sec)
        teste_5k_segundos = mm_ss_to_seconds(teste_5k_min, teste_5k_sec)

        dados = {
            "identificacao": {
                "idade_anos": idade_anos.value,
                "sexo": sexo.value,
                "altura_cm": float(altura_cm.value),
                "peso_kg": float(peso_kg.value),
                "profissao_atividade": profissao_atividade.value,
                "horas_sono_noite": float(horas_sono_noite.value),
                "estresse_diario_0_10": estresse_diario.value
            },
            "historico_corrida": {
                "tempo_corrida_meses": tempo_corrida_meses.value,
                "volume_semanal_km": float(volume_semanal_km.value),
                "treinos_semana": treinos_semana.value,
                "ritmo_facil_seg_km": ritmo_facil_seg_km.value,
                "ritmo_moderado_seg_km": ritmo_moderado_seg_km.value,
                "melhor_5k_segundos": melhor_5k_segundos,
                "melhor_10k_segundos": melhor_10k_segundos,
                "melhor_21k_segundos": melhor_21k_segundos,
                "melhor_42k_segundos": melhor_42k_segundos,
                "provas_12_meses": bool(provas_12_meses.value)
            },
            "historico_lesoes_saude": {
                "lesao_tendinite": bool(lesao_tendinite.value),
                "lesao_fascite": bool(lesao_fascite.value),
                "lesao_itbs": bool(lesao_itbs.value),
                "lesao_dor_lombar": bool(lesao_dor_lombar.value),
                "lesao_fratura_estresse": bool(lesao_fratura_estresse.value),
                "lesao_outra": bool(lesao_outra.value),
                "frequencia_lesoes_ano": frequencia_lesoes_ano.value,
                "condicao_medica": condicao_medica.value,
                "usa_medicacao": bool(usa_medicacao.value)
            },
            "dados_fisiologicos_testes": {
                "fc_reposo_bpm": fc_reposo_bpm.value,
                "fc_max_bpm": fc_max_bpm.value,
                "teste_1k_segundos": teste_1k_segundos,
                "teste_3k_segundos": teste_3k_segundos,
                "teste_5k_segundos": teste_5k_segundos,
                "disp_frequencia_cardiaca": bool(disp_fc.value),
                "disp_potencia": bool(disp_potencia.value),
                "disp_cadencia": bool(disp_cadencia.value),
                "disp_gps": bool(disp_gps.value)
            },
            "objetivos": {
                "objetivo_principal": objetivo_principal.value,
                "meta_distancia": meta_distancia.value,
                "reducao_tempo_minutos": float(reducao_tempo_min.value),
                "data_proxima_prova": data_proxima_prova.value,
                "motivacao_0_10": motivacao_0_10.value
            },
            "restricoes_disponibilidade": {
                "dias_treino_semana": dias_treino_semana.value,
                "tempo_max_sessao_min": float(tempo_max_sessao_min.value),
                "acesso_academia": bool(acesso_academia.value),
                "preferencia_treino": [preferencia_treino.value],
                "aderencia_historica": aderencia_historica.value
            },
            "classificacao_automatica": {
                "nivel_percebido_0_10": nivel_percebido.value,
                "disponibilidade_energetica_0_10": disponibilidade_energetica.value,
                "tolerancia_intenso_0_10": tolerancia_intenso.value,
                "contraindicacao_alta_intensidade": bool(contraindicacao_alta_intensidade.value)
            }
        }

        classificacao = classificar_corredor(
            volume_semanal_km=float(volume_semanal_km.value),
            melhor_5k_segundos=melhor_5k_segundos
        )
        dados["rotulo_classificacao"] = classificacao

        with open("corredor_dados.json", "w", encoding="utf-8") as f:
            json.dump(dados, f, ensure_ascii=False, indent=2)

        print("Dados salvos em 'corredor_dados.json'.")
        print(f"Classificação automática do corredor: {classificacao}")

botao_salvar.on_click(on_salvar_clicked)

formulario = VBox([
    box_identificacao,
    box_historico_corrida,
    box_lesoes_saude,
    box_dados_fisio,
    box_objetivos,
    box_restricoes,
    box_classificacao_auto,
    botao_salvar,
    output
])

display(formulario)
